In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def hawkes_simulation(mu, alpha, beta, T, dt):
    t = [0]
    N = [1]
    while t[-1] < T:
        u = np.random.uniform(0, 1)
        lambda_t = mu + alpha * np.sum(np.exp(-beta * (t[-1] - np.array(t[:-1]))))
        t_new = t[-1] - (1 / lambda_t) * np.log(u)
        if t_new <= T:
            t.append(t_new)
            N.append(N[-1] + 1)
    return np.array(t), np.array(N)

T = 10
dt = 0.01
mu = 1
alpha = 1
beta = 0.5

t, N = hawkes_simulation(mu, alpha, beta, T, dt)

plt.plot(t, N, 'o')
plt.xlabel('Time (t)')
plt.ylabel('Number of events (N)')
plt.show()


In [ ]:
import numpy as np

def hawkes_process(mu, alpha, beta, T, n_events=1):
    """Simulate a Hawkes process.
    
    Parameters
    ----------
    mu : float
        The background intensity rate.
    alpha : float
        The self-excitement parameter.
    beta : float
        The decay parameter.
    T : float
        The end time of the simulation.
    n_events : int, optional
        The number of events to simulate, by default 1.
    
    Returns
    -------
    events : numpy array
        The event times.
    """
    events = []
    t = 0
    for i in range(n_events):
        u = np.random.uniform(0, 1)
        t += -np.log(u) / (mu + np.sum([alpha * np.exp(-beta * (t - ti)) for ti in events]))
        if t > T:
            break
        events.append(t)
    return np.array(events)


In [ ]:
for i in range(10):
    print(hawkes_process(1, 0.5, 0.001, 10, n_events=1))

In [ ]:
import numpy as np
from scipy.optimize import minimize

def intensity(t, mu, alpha, beta, history):
    return mu + np.sum(alpha * np.exp(-beta * (t - history)), axis=0)

def log_likelihood(params, events):
    mu, alpha, beta = params[:3], params[3:-1].reshape(-1, 1), params[-1]
    N = events.shape[0]
    t = 0
    log_likelihood = 0
    history = []
    for i in range(N):
        dt = events[i] - t
        log_likelihood += np.log(intensity(t, mu, alpha, beta, history)) - intensity(events[i], mu, alpha, beta, history) * dt
        history.append(events[i])
        t = events[i]
    return -log_likelihood

def fit(events):
    D = events.shape[1]
    params_0 = np.concatenate((np.ones(D), np.ones(D), np.ones(1)))
    bounds = [(0, None)] * (2 * D + 1)
    result = minimize(log_likelihood, params_0, args=(events,), bounds=bounds)
    return result.x

events = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]).T
params = fit(events)
print(params)


In [ ]:
import numpy as np
from scipy.optimize import minimize

def hawkes_intensity(t, mu, alpha, beta, events):
    intensity = mu + np.sum(alpha[:, None] * np.exp(-beta[:, None] * (t - events)), axis=0)
    return intensity

def negative_log_likelihood(params, events):
    T = events[-1][-1]
    dt = 0.01
    N = int(T/dt)
    t = np.linspace(0, T, N)
    mu, alpha, beta = params[0], params[1:-1:2], params[2:-1:2]
    intensity = hawkes_intensity(t, mu, alpha, beta, events)
    log_likelihood = -np.sum(np.log(intensity)) * dt
    for event_seq in events:
        for t_i in event_seq:
            log_likelihood += np.log(mu + np.sum(alpha * np.exp(-beta * (t_i - event_seq))))
    return -log_likelihood

def fit_hawkes(events):
    num_sequences = len(events)
    num_events = np.sum([len(seq) for seq in events])
    dim = 2 * num_sequences + 1
    x0 = np.ones(dim)
    result = minimize(negative_log_likelihood, x0, args=(events,), method='L-BFGS-B')
    return result.x

events = [[1, 2, 3, 4], [5, 6, 7]]
params = fit_hawkes(events)
mu, alpha, beta = params[0], params[1:-1:2], params[2:-1:2]
